## Mount on google Colab

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


## Install Packages

In [ ]:
pip install yfinance   #install yfinance API

## API Connection Definition

In [6]:
import requests
import json


def get_ratios(ticker, limit, period):
  category = 'ratios'
  key = 'ed16b7e0331142766fdb030434e08a89'
  link = 'https://financialmodelingprep.com/api/v3/{category}/{ticker}?period={period}&limit={limit}&apikey={key}'
  url = link.format(category = category, ticker = ticker, period = period, limit = limit, key = key )
  response = requests.get(url)        # To execute get request 
  result = response.text  
  data = json.loads(result)       # convert json response to list 
 
  return data

In [7]:
# get_industry information
def get_industry(ticker):
  category = 'profile'
  key = 'ed16b7e0331142766fdb030434e08a89'
  profile_link = 'https://financialmodelingprep.com/api/v3/{category}/{ticker}?apikey={key}'
  profile_url = profile_link.format(category = category, ticker = ticker,key = key )
  response = requests.get(profile_url)        # To execute get request  
  result = response.text  
  profile_data = json.loads(result) 
  industry = profile_data[0]['sector']
  return industry

In [ ]:
# Take too much time for getting price. Change another method
'''
def get_movements(ticker, start, end):
  category = 'historical-price-full'
  key = 'ed16b7e0331142766fdb030434e08a89'
  price_link = 'https://financialmodelingprep.com/api/v3/{category}/{ticker}?from={start}&to={end}&apikey={key}'
  price_url = price_link.format(category = category, ticker = ticker,start = start, end = end, key = key )
  response = requests.get(price_url)        # To execute get request 
  #print(response.status_code)     # To print http response code  
  result = response.text  
  price_data = json.loads(result) 
  #price_data[0]

  Movements = price_data['historical'][0]['adjClose'] - price_data['historical'][-1]['adjClose']
  if Movements >= 1:
    RorF = 1
  else:
    RorF = 0
  return RorF

'''

## Upload s&p 500 elements list

In [8]:
import pandas as pd
from google.colab import files
import io

# Upload stock name
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['Stocks in the SP 500 Index.csv']))

Saving Stocks in the SP 500 Index.csv to Stocks in the SP 500 Index.csv


## Create History

## Start extracting Data from API

In [9]:
del (Database) # reset database

NameError: ignored

In [ ]:

import yfinance as yf
from __future__ import print_function
import sys


ticker_list = df.Symbol.to_list()


limit = 80
period = 'quarter'
ticker_num = 0


for n in ticker_list:
  ticker_num += 1     #Check loop status
  try:
    ratios_data = get_ratios(ticker = n, limit = limit, period = period)    # Get Ratios Data 
    test = ratios_data[0]    # For checking purpose, if null then continue
  except:
    continue


  # Status: Get the stock data price and assign trend value to Ratios dataframe

  Data = pd.DataFrame(ratios_data)


  data = yf.download(n , start="2002-02-12", end="2022-02-12")     #Get Price Data from yfinance (faster)
  data2 = data['Adj Close']

  price_list = []

  ticker_period = 0

  for i in range(len(Data)):
    ticker_period += 1               # Check ticker period 
    try: 
      price = data2.loc[Data['date'][i]]
    except:
      price = data2.iloc[data2.index.get_loc(Data['date'][i],method='nearest')]   #Get the nearest Adjusted Close Price
    
    price_list.append(price)



  Data['price'] = price_list
  Data['trend'] = Data['price'] > Data['price'].shift(-1)
  Data['trend'] = Data['trend'].astype(int)
  Data['trend'] = Data['trend'].shift(1)                  # Assign 1 and 0 to movements
  Data['industry']= get_industry(n)      #Assign industry
  Data = Data.iloc[1:,:]

  try:
    Database = Database.append(Data)    # Combine each stock data 

  except:
    Database = Data

  print('Now is number', ticker_num, 'Total Period:', ticker_period, end=' ')
  print(len(Database))
  sys.stdout.flush()

  
# Take 20 mins

In [11]:
Database

,symbol,date,period,currentRatio,quickRatio,cashRatio,daysOfSalesOutstanding,daysOfInventoryOutstanding,operatingCycle,daysOfPayablesOutstanding,cashConversionCycle,grossProfitMargin,operatingProfitMargin,pretaxProfitMargin,netProfitMargin,effectiveTaxRate,returnOnAssets,returnOnEquity,returnOnCapitalEmployed,netIncomePerEBT,ebtPerEbit,ebitPerRevenue,debtRatio,debtEquityRatio,longTermDebtToCapitalization,totalDebtToCapitalization,interestCoverage,cashFlowToDebtRatio,companyEquityMultiplier,receivablesTurnover,payablesTurnover,inventoryTurnover,fixedAssetTurnover,assetTurnover,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,payoutRatio,operatingCashFlowSalesRatio,freeCashFlowOperatingCashFlowRatio,cashFlowCoverageRatios,shortTermCoverageRatios,capitalExpenditureCoverageRatio,dividendPaidAndCapexCoverageRatio,dividendPayoutRatio,priceBookValueRatio,priceToBookRatio,priceToSalesRatio,priceEarningsRatio,priceToFreeCashFlowsRatio,priceToOperatingCashFlowsRatio,priceCashFlowRatio,priceEarningsToGrowthRatio,priceSalesRatio,dividendYield,enterpriseValueMultiple,priceFairValue,price,trend,industry
1,AAPL,2021-09-25,Q4,1.074553,0.909660,0.278449,225.524112,12.289877,67.898562,102.284273,-34.385711,0.421953,0.285341,0.278887,0.246533,0.116010,0.058550,0.325741,0.105471,0.883990,0.977382,0.285341,0.820257,4.563512,0.633615,0.664074,35.395833,0.161964,5.563512,1.618452,0.879901,7.3231,2.11359,0.237492,1.225199,1.029713,3.799269,0.177120,0.242322,0.840446,0.161964,1.293794,-6.267453,48.441247,0.17712,38.394164,38.394164,29.058155,29.466787,142.680555,119.915237,119.915237,-6.433582,29.058155,0.001503,93.354149,38.394164,146.519211,1.0,Technology
2,AAPL,2021-06-26,Q3,1.061891,0.887243,0.315998,151.980991,10.0916,47.566365,78.754629,-31.188264,0.432927,0.296264,0.299248,0.267014,0.107719,0.065923,0.338270,0.108634,0.892281,1.010072,0.296264,0.805118,4.131301,0.621954,0.65454,36.279699,0.173198,5.131301,2.401616,1.14279,8.918308,2.10887,0.246889,1.268479,1.142617,3.710062,0.173243,0.259032,0.900777,0.173198,1.315169,-10.078356,12.600956,0.173243,34.435837,34.435837,27.181958,25.449958,116.495742,104.936740,104.936740,-3.588444,27.181958,0.001702,82.583672,34.435837,132.548294,1.0,Technology
3,AAPL,2021-03-27,Q2,1.141749,0.966960,0.361574,134.601491,9.119697,42.309106,70.118047,-27.808941,0.425065,0.307008,0.312679,0.263775,0.156403,0.070086,0.341583,0.119178,0.843597,1.018471,0.307008,0.794820,3.873775,0.610966,0.637476,41.049254,0.197139,4.873775,2.711708,1.28355,9.868749,2.369007,0.265703,1.431404,1.295970,4.168329,0.145874,0.267693,0.905383,0.197139,1.844267,-10.568973,20.357385,0.145874,29.354546,29.354546,22.667985,21.484224,93.528409,84.679071,84.679071,-1.259420,22.667985,0.001697,67.153816,29.354546,120.493835,1.0,Technology
4,AAPL,2020-12-26,Q1,1.163003,1.022180,0.271759,192.000108,6.6691,54.011593,85.621433,-31.60984,0.397778,0.300918,0.301322,0.258034,0.143661,0.081216,0.434208,0.151363,0.856339,1.001342,0.300918,0.812955,4.346309,0.599867,0.628512,52.561129,0.345965,5.346309,1.901041,1.051139,13.495073,2.937785,0.314751,2.288912,2.082241,4.536490,0.125648,0.347841,0.909708,0.345965,3.037377,-11.075143,343.035398,0.125648,34.955023,34.955023,20.772454,20.125730,65.645619,59.718325,59.718325,0.155136,20.772454,0.001561,64.823752,34.955023,135.679626,0.0,Technology
5,AAPL,2020-09-26,Q4,1.363604,1.218195,0.360710,211.249575,9.135195,61.224131,95.144592,-33.920462,0.381604,0.228369,0.230316,0.195879,0.149520,0.039128,0.193958,0.067621,0.850480,1.008528,0.228369,0.798267,3.957039,0.601606,0.632462,23.304416,0.183002,4.957039,1.727814,0.945929,9.852007,1.759724,0.199754,1.206264,1.101678,5.331517,0.277046,0.318031,0.913297,0.183002,1.494371,-11.533632,11.914302,0.277046,29.312199,29.312199,29.602612,37.781697,101.917294,93.080763,93.080763,2.817435,29.602612,0.001833,109.099621,29.312199,111.258118,1.0,Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [12]:
Database.industry.unique()

array(['Technology', 'Communication Services', 'Consumer Cyclical',
       'Financial Services', 'Healthcare', 'Consumer Defensive', 'Energy',
       'Industrials', 'Basic Materials', 'Utilities', 'Real Estate'],
      dtype=object)

In [ ]:
## Some industries appear null, try other methods
'''
# Create industry dictionary
ticker_list = df['Symbol'].to_list()
industry_dict = {}
for i in ticker_list:
  try:
    industry = get_industry(i)
  except:
    continue
    
  industry_dict[i] = industry
industry_dict


# Assign industry to each ticker 
total_num = len(Database)
Database['industry']=[0 for i in range(len(Database))]

for i in range(total_num):
  name = str(Database.iloc[i:i+1,:]['symbol'].values[0])
  try:
    ticker_industry = industry_dict[name]
    Database['industry'][i] = ticker_industry
  except:
    continue

'''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


## Save to CSV

In [13]:
from google.colab import files
Database.to_csv('Data.csv')
files.download("Data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>